In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import functions as f

In [20]:
# Reading and cleaning dim_arrival_mode
dim_arrival_mode = spark.read.csv('I94MODE.csv', sep=',', header=True, inferSchema=True)
dim_arrival_mode.toPandas().head()

,ID,Mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [23]:
# Making headers and string to lower to avoid misspelling
dim_arrival_mode = dim_arrival_mode.toDF('code', 'mode')
dim_arrival_mode = dim_arrival_mode.withColumn('mode', f.lower(f.col('mode')))
dim_arrival_mode.toPandas()

,code,mode
0,1,air
1,2,sea
2,3,land
3,9,not reported


In [28]:
# Using .toPandas().to_csv to avoid folder creation (files are very small)
# dim_arrival_mode.write.csv('dim_arrival_mode.csv', header=True)
dim_arrival_mode.toPandas().to_csv('dim_arrival_mode.csv', header=True, index=False) 

In [24]:
# Reading and cleaning dim_visa
dim_visa = spark.read.csv('I94VISA.csv', sep=',', header=True, inferSchema=True)
dim_visa.toPandas().head()

,ID,Type
0,1,Business
1,2,Pleasure
2,3,Student


In [25]:
# Making headers and string to lower to avoid misspelling
dim_visa = dim_visa.toDF('code', 'type')
dim_visa = dim_visa.withColumn('type', f.lower(f.col('type')))
dim_visa.toPandas()

,code,type
0,1,business
1,2,pleasure
2,3,student


In [29]:
dim_visa.toPandas().to_csv('dim_visa.csv', header=True, index=False)

In [44]:
# Reading and cleaning dim_countries
dim_countries = spark.read.csv('I94CIT_I94RES.csv', sep=',', header=True, inferSchema=True)
dim_countries.toPandas().head()

,Code,I94CTRY
0,582,MEXICO
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [50]:
# Making headers to lower to avoid misspelling
dim_countries = dim_countries.toDF('code', 'name')
dim_countries.columns

['code', 'name']

In [46]:
dim_countries.toPandas().to_csv('dim_countries.csv', header=True, index=False)

In [153]:
# Reading and cleaning dim_us_states
states = spark.read.csv('I94ADDR.csv', sep=',', header=True, inferSchema=True)
states.toPandas().head()

,code,state
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [154]:
# Making headers and string to lower to avoid misspelling
states = dim_us_states.toDF('code', 'state_name')
states.columns

['code', 'state_name']

In [155]:
# Gathering city codes and names
city_code = spark.read.csv('I94PORT.csv', sep=',', header=True, inferSchema=True)
city_code.toPandas().head()

,ID,Port
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"


In [156]:
city_code = city_code.withColumn('name', f.split(city_code.Port, ', ')[0])
city_code = city_code.withColumn('state_code', f.split(city_code.Port, ', ')[1])
city_code = city_code.drop('Port').toDF('city_code', 'name', 'state_code')

city_code.toPandas().head()

,city_code,name,state_code
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [157]:
city_code.filter(f.col('state_code').isNull()).count()

78

In [161]:
dim_us_states = city_code.join(states, city_code.state_code == states.code, how='left')
dim_us_states = dim_us_states.drop('code')
dim_us_states.show(10)

+---------+--------------------+----------+----------+
|city_code|                name|state_code|state_name|
+---------+--------------------+----------+----------+
|      ALC|               ALCAN|        AK|    ALASKA|
|      ANC|           ANCHORAGE|        AK|    ALASKA|
|      BAR|BAKER AAF - BAKER...|        AK|    ALASKA|
|      DAC|       DALTONS CACHE|        AK|    ALASKA|
|      PIZ|DEW STATION PT LA...|        AK|    ALASKA|
|      DTH|        DUTCH HARBOR|        AK|    ALASKA|
|      EGL|               EAGLE|        AK|    ALASKA|
|      FRB|           FAIRBANKS|        AK|    ALASKA|
|      HOM|               HOMER|        AK|    ALASKA|
|      HYD|               HYDER|        AK|    ALASKA|
+---------+--------------------+----------+----------+
only showing top 10 rows



In [162]:
dim_us_states.groupBy(dim_us_states.state_code).count().show(60)

+-------------------+-----+
|         state_code|count|
+-------------------+-----+
|            BAHAMAS|    2|
|       SOUTH AFRICA|    1|
|           AR (BPS)|    1|
|                 AZ|   11|
|                 SC|    5|
|                 NS|    1|
|              JAPAN|    1|
|    SUGAR LAND ARPT|    1|
|                 LA|    5|
|                 MN|   22|
|                 NJ|    9|
|                 MX|    1|
|           CA (BPS)|    3|
|           FL #ARPT|    1|
|                 OR|    7|
|           NM (BPS)|    3|
|    EL DORADO #ARPT|    1|
|                SPN|    1|
|                 VA|    6|
|               null|   78|
|                 RI|    2|
|                 KY|    2|
|                 WY|    2|
|                 NH|    4|
|        SEOUL KOREA|    1|
|                 MI|   16|
|             CANADA|   10|
|                 NV|    4|
|VT (BP - SECTOR HQ)|    1|
|           ME (BPS)|    1|
|                 WI|    6|
|                 ID|    4|
|           COLOMBIA